In [6]:
import pandas as pd # Import the pandas library for data manipulation and analysis

# Load the dataset from a CSV file into a pandas DataFrame
a_df = pd.read_csv("combined_collisions_v3.csv")

# Save the DataFrame as a Parquet file for efficient storage and faster I/O
a_df.to_parquet("combined_collisions_v3.parquet", engine="pyarrow", index=False)

In [7]:
import pandas as pd # Import pandas for data handling and analysis

# Read the dataset from a Parquet file into a DataFrame
pa_df = pd.read_parquet("combined_collisions_v3.parquet")

# Preview the first 5 rows of the DataFrame
print(pa_df.head())

# Select and display specific columns of interest: Longitude, Latitude, Accident Severity, Date, Time and whether the accident took place in London or not
# Also deal with the outcomes: fatal, serious or slight
print(pa_df[["Longitude", "Latitude", "Accident_Severity", "Date", "Time", "in_london", "fatal", "serious", "slight"]].head())

# Iterate through the first 5 rows of the DataFrame 
# Print out the Date, Accident Severity and the outcome (fatal, serious and slight)
for _, row in pa_df.head(5).iterrows():
    print(row["Date"], row["Accident_Severity"], row["fatal"], row["serious"], row["slight"])

   Unnamed: 0 Accident_Index  Location_Easting_OSGR  Location_Northing_OSGR  \
0           0  200501BS00024               524700.0                180880.0   
1           1  200501BS00032               524470.0                180980.0   
2           2  200501BS70004               524870.0                181880.0   
3           3  200501BS70009               525840.0                177020.0   
4           4  200501BS70010               526940.0                177460.0   

   Longitude   Latitude  Police_Force  Accident_Severity  Number_of_Vehicles  \
0  -0.204346  51.513039             1                  3                   2   
1  -0.207623  51.513988             1                  3                   2   
2  -0.201543  51.521988             1                  3                   2   
3  -0.189301  51.478096             1                  3                   2   
4  -0.173312  51.481804             1                  3                   2   

   Number_of_Casualties  ... aadf_CLT_CBYPAS

In [8]:
import pandas as pd # Import pandas for data manipulation and analysis

# Load the dataset from a Parquet file into a DataFrame
pa_df = pd.read_parquet('combined_collisions_v3.parquet')

# Calculate the minimum and maximum values for geographic coordinates
minimum_latitude = pa_df['Latitude'].min()
maximum_latitude = pa_df['Latitude'].max()
minimum_longitude = pa_df['Longitude'].min()
maximum_longitude = pa_df['Longitude'].max()

# Print the results in a clear, labeled format
print(f"Minimum Latitude: {minimum_latitude}")
print(f"Maximum Latitude: {maximum_latitude}")
print(f"Minimum Longitude: {minimum_longitude}")
print(f"Maximum Longitude: {maximum_longitude}")

Minimum Latitude: 49.91443
Maximum Latitude: 60.344565
Minimum Longitude: -7.504121
Maximum Longitude: 1.76201


In [9]:
pip install geopy

In [10]:
import time # Import time to add pauses between API requests
from geopy.geocoders import Nominatim # Import Nominatim geocoder from geopy
import pandas as pd # Import pandas for data handling and export

boroughs = [ # List of all London boroughs to geocode
    "Barking and Dagenham", "Barnet", "Bexley", "Brent", "Bromley", "Camden", "Croydon", 
    "Ealing", "Enfield", "Greenwich", "Hackney", "Hammersmith and Fulham", "Haringey", 
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington", "Kensington and Chelsea", 
    "Kingston upon Thames", "Lambeth", "Lewisham", "Merton", "Newham", "Redbridge", 
    "Richmond upon Thames", "Southwark", "Sutton", "Tower Hamlets", "Waltham Forest", 
    "Wandsworth", "Westminster", "City of London"
]

geolocator = Nominatim(user_agent="london_boroughs_locator") # Initialise the geolocator with a custom user agent for place identification
borough_coordinates = {} # Dictionary to store borough names and their coordinates

for b in boroughs: # Loop through each borough and attempt to geocode it
    try:
        location = geolocator.geocode(f"{b}, London, UK", timeout=10) # Query Nominatim for the borough location (timeout to prevent server crashing)
        if location: # If a location is found, store latitude and longitude 
            borough_coordinates[b] = (location.latitude, location.longitude)
        else: # If not found, store None placeholders
            borough_coordinates[b] = (None, None)
    except Exception as e: # Handle errors gracefully and log the borough that failed
        print(f"Error with {b}: {e}") 
        borough_coordinates[b] = (None, None)
    
    time.sleep(1) # Pause for 1 second between requests to respect Nominatim’s usage policy

lb_df = pd.DataFrame(borough_coordinates.items(), columns=['Borough', 'Coordinates']) # Convert the dictionary into a DataFrame
lb_df[['Latitude', 'Longitude']] = pd.DataFrame(lb_df['Coordinates'].tolist(), index=lb_df.index) # Split the tuple in 'Coordinates' into separate Latitude and Longitude columns
lb_df.drop(columns=['Coordinates'], inplace=True) # Drop the original 'Coordinates' column 

lb_df.to_parquet("london_boroughs_coordinates.parquet", index=False) # Save the DataFrame to a Parquet file for efficient storage and reproducibility
print("The coordinates have been saved to london_boroughs_coordinates.parquet")

The coordinates have been saved to london_boroughs_coordinates.parquet


In [11]:
import pandas as pd

# Load the dataset from a Parquet file into a DataFrame
df = pd.read_parquet('london_boroughs_coordinates.parquet')

# Compute the minimum and maximum values for latitude and longitude based on the boroughs
minimum_latitude = df['Latitude'].min()
maximum_latitude = df['Latitude'].max()
minimum_longitude = df['Longitude'].min()
maximum_longitude = df['Longitude'].max()

print(f"Minimum Latitude: {minimum_latitude}")
print(f"Maximum Latitude: {maximum_latitude}")
print(f"Minimum Longitude: {minimum_longitude}")
print(f"Maximum Longitude: {maximum_longitude}")

Minimum Latitude: 51.3574645
Maximum Latitude: 51.65309
Minimum Longitude: -0.4483349
Maximum Longitude: 0.2498128


In [12]:
!python -m pip install --upgrade pip

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.1
    Uninstalling pip-25.1:
      Successfully uninstalled pip-25.1
